In [4]:
import pandas as pd
import os 
from src.files import XMLFile, FASTAFile
from utils import * 
from src.genome import ReferenceGenome

%load_ext autoreload 
%autoreload 2

In [4]:
genome_metadata_df = load_genome_metadata(refseq_only=True)

In [32]:
os.listdir('../data/refseq')
ref = ReferenceGenome('../data/refseq/GCF_000024365.1_genomic.gbff', genome_id='GCF_000024365.1')

In [33]:
df = FASTAFile('../data/prodigal/GCF_000024365.1_protein.faa').to_df(prodigal_output=True)

results_df = ref.search(df)

ReferenceGenome.search: 100%|██████████| 5465/5465 [00:55<00:00, 98.54it/s] 


ReferenceGenome._merge: 141 entries in the query DataFrame had no valid hits in the reference.


In [36]:
weird_df = results_df[(results_df.ref_feature == 'CDS') & (results_df.ref_stop != results_df.stop) & (results_df.strand == 1)][['ref_seq', 'ref_start', 'start', 'ref_stop', 'stop', 'ref_product', 'ref_pseudo', 'stop_codon', 'ref_stop_codon']]

for seq in weird_df.ref_seq:
    print(seq)


MAKPYPKEFRDDVVAVARKGQATLTQIAKDFGISEGSLANWMKQADIKDGRRPGLTDVDRAELRELKKRNRLLEQENEVLRRAAAYLSQANLPKIVFPLVREMAATGAPIRVPVAVTLRVLGLSRAGYYKWLSDPVCQRDYDDAHLIDKLYDLHGDDATLGYRFLTDELEDEHGIQVGENRVHRLCRIAGITASHHKKRSKTGSTGPAPHDDLLAVVDEHGVVRHEFLADAPNKVWLWDISEHPTREGKLYICAIKDVWSNKIVGYSIDSRMKSSLARAAMRNAIALRSPAGTVCHSDRGGQFRAKRTQRLLANNDLVGSMGRSHGAGDNASMESFFSLLQKNVLNTRRWDTRDDLRLAIVTWIETKYNRRRRQRALGKLTPVEFEMIYKAAEAA
nan
nan
nan
nan
MPAPRQYDQETRDRAVRMYRERRSSHPAESAIQARRQVGALIDVKSETLRGWIERSEIDAGVRPGTTSDEHTELVRLRREVAELRRANEILRTASAFFGSGGARPPTQVICAYIDAHKHRFGVEPICRVLSEHAMPIAPSSYYAHRRRPVSPALRAEAHLVNTVIDLHREHHGLYGVRKMWHLMTQEGHLVGRDQVARLMTIAGLRGVRRGDHRTTTTRRRAADDRPRPVDLINRDWAAPTRPDQWWVADFTYVWTPAGFCYVSFVVDVYSRRILGWRVASSMTTALVAAALDQALFTRRRTDTRFTSTGLVHHSDAGSQYTSIAYTTALVDAGIAPSIGSVGDALDNALMESTIGLYKTELIDRYRRSWAGPQQVETATAGWVHWFNTRRLHSSIDYRPPVTYEQLYRQQTSAPAGEAA
MAKPYPKEFRDDVVAVARKGQATLTQIAKDVGISEGSLANWMKQADIEDGRRPGLTDVDRAELRELKKRNRLLEQENEVLRRAAAYLSQANLPKIVFPLVREMAATGAPIRVPVAVTLRVLGLSRAGYYKWLSDPVCQRDYDDAHLIDKLYDLHGDDATLGYRFLTD